In [1]:
# Dependencies
import pandas as pd
import os
from bs4 import BeautifulSoup as bs
import time
import io
import requests
import hashlib
import sqlite3 as lite
import sys
from PIL import Image
from numpy import genfromtxt
from datetime import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, Session

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Date
import urllib

In [14]:
inputSearch="seagull call"
query = inputSearch
query = query.replace(' ', '+')
URL = f"https://google.com/search?q={query}"
query

'seagull+call'

In [3]:
# desktop user-agent
# USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
# mobile user-agent
MOBILE_USER_AGENT = "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"

In [4]:
headers = {"user-agent" : USER_AGENT}
resp = requests.get(URL, headers=headers)

In [5]:
if resp.status_code == 200:
    soup = bs(resp.content, "html.parser")

In [6]:
results = []
for g in soup.find_all('div', class_='r'):
    anchors = g.find_all('a')
    if anchors:
        link = anchors[0]['href']
        title = g.find('h3').text
        item = {
            "title": title,
            "link": link
        }
        results.append(item)
print(results)

[{'title': 'Herring Gull Sounds, All About Birds, Cornell Lab of Ornithology', 'link': 'https://www.allaboutbirds.org/guide/Herring_Gull/sounds'}, {'title': 'What is the meaning of all the sounds seagulls make? - Quora', 'link': 'https://www.quora.com/What-is-the-meaning-of-all-the-sounds-seagulls-make'}, {'title': 'Seagull Calling Contest | BirdNote', 'link': 'https://www.birdnote.org/show/seagull-calling-contest'}, {'title': 'What Gulls Say – Earbirding', 'link': 'http://earbirding.com/blog/archives/3994'}, {'title': 'Long Calls of Gulls – Earbirding', 'link': 'http://earbirding.com/blog/archives/4538'}, {'title': 'Seagull or Gull: Who Really Cares? | Audubon', 'link': 'https://www.audubon.org/news/seagull-or-gull-who-really-cares'}, {'title': 'Seagull Call Sounds | Effects | Sound Bites | Sound Clips from ...', 'link': 'http://soundbible.com/192-Seagull-Call.html'}, {'title': 'Audubon Bird Call : Whistles Musical ... - Amazon.com', 'link': 'https://www.amazon.com/Audubon-RE2473-Bird

In [7]:
df = pd.DataFrame(results)
df

,title,link
0,"Herring Gull Sounds, All About Birds, Cornell ...",https://www.allaboutbirds.org/guide/Herring_Gu...
1,What is the meaning of all the sounds seagulls...,https://www.quora.com/What-is-the-meaning-of-a...
2,Seagull Calling Contest | BirdNote,https://www.birdnote.org/show/seagull-calling-...
3,What Gulls Say – Earbirding,http://earbirding.com/blog/archives/3994
4,Long Calls of Gulls – Earbirding,http://earbirding.com/blog/archives/4538
5,Seagull or Gull: Who Really Cares? | Audubon,https://www.audubon.org/news/seagull-or-gull-w...
6,Seagull Call Sounds | Effects | Sound Bites | ...,http://soundbible.com/192-Seagull-Call.html
7,Audubon Bird Call : Whistles Musical ... - Ama...,https://www.amazon.com/Audubon-RE2473-Bird-Cal...


In [8]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>link</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Herring Gull Sounds, All About Birds, Cornell Lab of Ornithology</td>\n      <td>https://www.allaboutbirds.org/guide/Herring_Gull/sounds</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>What is the meaning of all the sounds seagulls make? - Quora</td>\n      <td>https://www.quora.com/What-is-the-meaning-of-all-the-sounds-seagulls-make</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Seagull Calling Contest | BirdNote</td>\n      <td>https://www.birdnote.org/show/seagull-calling-contest</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>What Gulls Say – Earbirding</td>\n      <td>http://earbirding.com/blog/archives/3994</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Long Calls of Gulls – Earbirding</td>\n      <td>http://earbirding.com/blog/archive

# Read sqlite database that contains Oregon birds information

In [9]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base 
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [10]:
# exstract list of birds English names
database_path = "data/birdDB.sqlite"

In [11]:
conn = lite.connect(database_path)

In [12]:
cur = conn.cursor()

In [13]:
#  Read unique bird names from uniqueName table 
query = "SELECT * FROM uniqueNames;"

df = pd.read_sql_query(query,conn)
df

,index,uniqueName,imageLocation
0,0,Acorn Woodpecker,./images\acorn_woodpecker\1651956c1e.jpg
1,1,American Bushtit,./images\american_bushtit\23de54f7b6.jpg
2,2,American Cliff Swallow,./images\american_cliff_swallow\4d50746d92.jpg
3,3,American Crow,./images\american_crow\4aa703f5b0.jpg
4,4,American Grey Flycatcher,./images\american_grey_flycatcher\bc34237459.jpg
...,...,...,...
112,112,Wilson's Snipe,./images\wilson's_snipe\e2279f5393.jpg
113,113,Wood Duck,./images\wood_duck\7173954478.jpg
114,114,Wrentit,./images\wrentit\a43f0b1268.jpg
115,115,Yellow Rail,./images\yellow_rail\d5d6b7133e.jpg
